In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [5]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-12 17:54:09--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.02s   

2025-10-12 17:54:09 (4.21 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [7]:
df = pd.read_csv('course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [9]:
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


## Data preparation

- Check if the missing values are presented in the features.
- If there are missing values:
    - For caterogiral features, replace them with 'NA'
    - For numerical features, replace with with 0.0

In [15]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [13]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [17]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [152]:
#cat_cols = features.select_dtypes(include=["object", "category"]).columns.tolist()
#num_cols = features.select_dtypes(include=[np.number]).columns.tolist()

numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
categorical = ['lead_source', 'industry', 'employment_status', 'location']

df[numerical] = df[numerical].fillna(0.0)
df[categorical] = df[categorical].fillna('NA')

In [154]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

## Question 1

What is the most frequent observation (mode) for the column industry?

- NA
- technology
- healthcare
- retail


In [157]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

## Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

- interaction_count and lead_score
- number_of_courses_viewed and lead_score
- number_of_courses_viewed and interaction_count
- annual_income and interaction_count

Note: Only consider the pairs above when answering this question.

In [160]:
corr_matrix = df[numerical].corr()
corr_matrix

# Answer -- annual_income and interaction_count

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [162]:
df[numerical].corrwith(df['lead_score'])

number_of_courses_viewed   -0.004879
annual_income               0.015610
interaction_count           0.009888
lead_score                  1.000000
dtype: float64

## Split the data

- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
- Make sure that the target value y is not in your dataframe.

In [165]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)
print(len(df_full_train), len(df_test))

1169 293


In [167]:
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42)
print(len(df_train), len(df_val), len(df_test))

876 293 293


In [169]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [171]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [173]:
len(y_train), len(y_val), len(y_test)

(876, 293, 293)

In [175]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

## Question 3

- Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using round(score, 2).

Which of these variables has the biggest mutual information score?

- industry
- location
- lead_source
- employment_status

In [178]:
from sklearn.metrics import mutual_info_score

# mutual_info_score(y_train, df_train[categorical])

for c in categorical:
    mi = round(mutual_info_score(y_train, df_train[c]),2)

    print(c, mi)

## Answer -- Lead_Source
# Wtihout rounding
#lead_source 0.03539624379726594
#industry 0.011574521435657112
#employment_status 0.012937677269442782
#location 0.004464157884038034

lead_source 0.04
industry 0.01
employment_status 0.01
location 0.0


## Question 4

- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

Calculate the accuracy on the validation dataset and round it to 2 decimal digits. What accuracy did you get?

- 0.64
- 0.74
- 0.84
- 0.94

In [181]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=True)

In [183]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [185]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [187]:
y_pred = model.predict_proba(X_val)[:,1]   ## soft predictions
y_pred

array([0.61192161, 0.79982616, 0.53021342, 0.47131479, 0.5706613 ,
       0.44227166, 0.87127669, 0.84883114, 0.83290036, 0.614978  ,
       0.54968025, 0.78153087, 0.69039784, 0.77017121, 0.52659438,
       0.91706424, 0.53170633, 0.42123047, 0.30146454, 0.84881582,
       0.79488652, 0.73670373, 0.44527209, 0.64838382, 0.41768818,
       0.75393417, 0.90166115, 0.33903047, 0.43181429, 0.9680681 ,
       0.92018714, 0.37487986, 0.65230098, 0.90650057, 0.75164115,
       0.6420212 , 0.82250074, 0.83375553, 0.65911598, 0.30978852,
       0.78942264, 0.35546364, 0.96517758, 0.63389304, 0.51274193,
       0.53230532, 0.82287784, 0.744074  , 0.73452312, 0.68955215,
       0.46964441, 0.84539251, 0.55635242, 0.92637871, 0.65258021,
       0.61526271, 0.63816993, 0.28304016, 0.48049823, 0.57890616,
       0.3549734 , 0.62175049, 0.38960775, 0.61156055, 0.85304277,
       0.75430135, 0.89185953, 0.71946457, 0.95387623, 0.89209517,
       0.75277086, 0.33850137, 0.61376593, 0.51622273, 0.64088

In [189]:
converted_decision = (y_pred >= 0.5)
converted_decision

array([ True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True, False,  True, False,  True,  True,
       False, False,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True, False, False,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True, False,  True,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False,  True,

In [191]:
(y_val == converted_decision).mean()

0.6996587030716723

In [193]:
from sklearn.metrics import accuracy_score

y_val_proba = model.predict_proba(X_val)[:, 1]
y_val_pred  = (y_val_proba >= 0.5)

acc = accuracy_score(y_val, y_val_pred)
print("Validation accuracy:", round(acc, 2))

Validation accuracy: 0.7


## Question 5
- Let's find the least useful feature using the feature elimination technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

- 'industry'
- 'employment_status'
- 'lead_score'

Note: The difference doesn't have to be positive.

In [200]:
feat_all = categorical + numerical  

def acc_with_features(cols):
    dv = DictVectorizer(sparse=True)
    X_tr = dv.fit_transform(df_train[cols].to_dict(orient="records"))
    X_va = dv.transform(df_val[cols].to_dict(orient="records"))
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_tr, y_train)
    return accuracy_score(y_val, model.predict(X_va))



In [202]:
baseline_acc = acc_with_features(feat_all)
print("Baseline accuracy (all features):", baseline_acc)

Baseline accuracy (all features): 0.6996587030716723


In [206]:
drops = {}
for col in feat_all:
    cols_wo = [c for c in feat_all if c != col]
    acc_wo = acc_with_features(cols_wo)
    drops[col] = baseline_acc - acc_wo  

drops_s = pd.Series(drops).sort_values(ascending=True)
print("\nAccuracy drop when removing each feature (sorted, least useful first):")
print(drops_s)

least_useful = drops_s.index[0]
print(f"\nLeast useful by this criterion: {least_useful} (drop={drops_s.iloc[0]:.4f})")


## Answer -- lead score


Accuracy drop when removing each feature (sorted, least useful first):
annual_income              -0.153584
location                   -0.010239
lead_score                 -0.006826
lead_source                -0.003413
industry                    0.000000
employment_status           0.003413
number_of_courses_viewed    0.143345
interaction_count           0.143345
dtype: float64

Least useful by this criterion: annual_income (drop=-0.1536)


## Question 6

- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these C leads to the best accuracy on the validation set?

- 0.01
- 0.1
- 1
- 10
- 100

Note: If there are multiple options, select the smallest C.



In [209]:
feat = categorical + numerical  # your lists

# One-hot once
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(df_train[feat].to_dict("records"))
X_val   = dv.transform(df_val[feat].to_dict("records"))

Cs = [0.01, 0.1, 1, 10, 100]
rows = []

for C in Cs:
    model = LogisticRegression(solver="liblinear", C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    acc = accuracy_score(y_val, model.predict(X_val))
    rows.append({"C": C, "val_accuracy": round(acc, 3)})

results = pd.DataFrame(rows)
print(results)

# Optional: show best C by accuracy
best = results.loc[results["val_accuracy"].idxmax()]
print(f"Best C: {best.C} with acc={best.val_accuracy:.3f}")

# Answer -- 0.01

        C  val_accuracy
0    0.01           0.7
1    0.10           0.7
2    1.00           0.7
3   10.00           0.7
4  100.00           0.7
Best C: 0.01 with acc=0.700
